In [1]:
import learning2read
from learning2read.b06 import *
from learning2read.proc import Procedure
from learning2read.preprocessing import TotalDataFrame
from importlib import reload

In [2]:
from scipy.sparse import csr_matrix

frame = Data("dattrain").sample(40000)

userlst = list(frame['User-ID'].unique())
isbnlst = list(frame['ISBN'].unique())

data = frame['Book-Rating'].tolist()
row = frame['User-ID'].astype(pd.api.types.CategoricalDtype(categories=userlst)).cat.codes
col = frame['ISBN'].astype(pd.api.types.CategoricalDtype(categories=isbnlst)).cat.codes
# col = frame.thing.astype('category', categories=thing_u).cat.codes
M = csr_matrix((data, (row, col)), shape=(len(userlst), len(isbnlst)))

len(userlst), len(isbnlst)

(17490, 29273)

**sklearn的NMF不支援missing value...**

In [3]:
# model = NMF(n_components = 3, init='random', random_state=0, tol=0.001)
# W = model.fit_transform(M)
# H = model.components_

[先用這個試試看](https://stackoverflow.com/questions/22767695/python-non-negative-matrix-factorization-that-handles-both-zeros-and-missing-dat)

In [4]:
import numpy as np
from scipy import linalg
from numpy import dot

class NMF:
    def __init__(self, X, latent_features, eps=1e-5):
        self.eps = eps
        self.latent_features = latent_features
        self.X = X.toarray()
        self.mask = np.sign(self.X)
        self.rows, self.columns = self.X.shape
        self.A = np.random.rand(self.rows, self.latent_features)
        self.A = np.maximum(self.A, self.eps)
        self.Y = linalg.lstsq(self.A, self.X)[0]
        self.Y = np.maximum(self.Y, self.eps)
        self.masked_X = self.mask * self.X
        self.X_est_prev = dot(self.A, self.Y)
        self.iters = 0
        
    def DoIter(self):
        top = dot(self.masked_X, self.Y.T)
        bottom = (dot((self.mask * dot(self.A, self.Y)), self.Y.T)) + self.eps
        self.A *= top / bottom
        self.A = np.maximum(self.A, self.eps)
        top = dot(self.A.T, self.masked_X)
        bottom = dot(self.A.T, self.mask * dot(self.A, self.Y)) + self.eps
        self.Y *= top / bottom
        self.Y = np.maximum(self.Y, self.eps)
        self.iters += 1
        
    def GetWH(self): return self.A, self.Y
    def Evaluate(self):
        X_est = dot(self.A, self.Y)
        err = self.mask * (self.X_est_prev - X_est)
        fit_residual = np.sqrt(np.sum(err ** 2))
        self.X_est_prev = X_est
        curRes = linalg.norm(self.mask * (self.X - X_est), ord = 'fro')
        return fit_residual, curRes


In [ ]:
alldata = Data("dattrain")
comp = alldata.loc[~alldata.isin(frame).all(1)]
isbnid = dict((j,i) for i,j in enumerate(isbnlst))
userid = dict((j,i) for i,j in enumerate(userlst))
comp = comp.loc[comp['ISBN'].isin(isbnid)]
comp = comp.loc[comp['User-ID'].isin(userid)]

def GetEval(W, H):
    comp['predicted'] = comp.apply(
        lambda r: min(10, int(np.dot(W[userid[r['User-ID']],:], H[:,isbnid[r['ISBN']]]) + 0.5)), axis=1)
    return np.array(comp.apply(lambda r: abs(r['Book-Rating'] - r['predicted']), axis=1)).sum() / comp.shape[0]

f = open('/tmp2/b06902021/ML/NMFresult', 'a')

for dimen in [3,5,10,20,40,70]:
    for seed in range(2):
        np.random.seed(seed)
        model = NMF(M, 20)
        W, H = model.GetWH()
        print(dimen, 0, model.Evaluate(), GetEval(W, H), file=f)
        f.flush()
        for i in range(8):
            model.DoIter()
            W, H = model.GetWH()
            print(dimen, i + 1, model.Evaluate(), GetEval(W, H), file=f)
            f.flush()
